In [1]:
import pandas as pd
import requests
import time
import geopandas as gpd
from dateutil.relativedelta import relativedelta
import datetime as dt
import math

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [3]:
# Read in new file
file_loc_raw = "../datasets/resale_hdb_price_raw_30sep.csv"
df_raw = pd.read_csv(file_loc_raw, parse_dates=["month", "lease_commence_date"])

df_raw["floor_area_sqft"] = df_raw["floor_area_sqm"] * 10.7639
df_raw["price_per_sqft"] = (
    df_raw["resale_price"] / df_raw["floor_area_sqft"]
)

# df_raw = df_raw.drop(["remaining_lease", "lease_commence_date"], axis=1)

In [4]:
df_raw.head(1)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,473.6116,489.852867


In [5]:
df_raw.shape

(190724, 13)

### Begin the synchronisation process

#### 1. Open the masterlist of HDB addresses. This will be used to perform a left join to the new data

In [6]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc, index_col=0, dtype={"postal": "object"}, parse_dates=["lease_commence_date"]
)

masterlist_df.head()

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254
1,ANG MO KIO,207,ANG MO KIO AVE 1,207,ANG MO KIO AVENUE 1,ANG MO KIO 22,560207,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29060.485578,38651.052977,1.365821,103.842848,Ang Mo Kio,874.305666,MRT,Red,9084.169820,ANG MO KIO PRIMARY SCHOOL,527.275749
2,ANG MO KIO,208,ANG MO KIO AVE 1,208,ANG MO KIO AVENUE 1,ANG MO KIO 22,560208,208 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29045.715075,38609.483079,1.365445,103.842715,Ang Mo Kio,908.966103,MRT,Red,9044.410980,ANG MO KIO PRIMARY SCHOOL,549.157654
3,ANG MO KIO,215,ANG MO KIO AVE 1,215,ANG MO KIO AVENUE 1,ANG MO KIO 22,560215,215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28924.303291,38732.591142,1.366558,103.841624,Mayflower,781.530965,MRT,Brown,9180.500545,ANG MO KIO PRIMARY SCHOOL,377.594832
4,ANG MO KIO,216,ANG MO KIO AVE 1,216,ANG MO KIO AVENUE 1,ANG MO KIO 22,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28911.052240,38692.616791,1.366197,103.841505,Mayflower,800.632187,MRT,Brown,9142.409400,ANG MO KIO PRIMARY SCHOOL,403.611794


#### 1a: First, check if there are any new addresses that appear in the new dataframe

In [8]:
df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge
187771,2024-08-01,SENGKANG,2 ROOM,457B,SENGKANG WEST RD,01 TO 03,38.0,2-room,2020-01-01,95 years 01 month,360000.0,409.0282,880.134915,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
189970,2024-09-01,SEMBAWANG,3 ROOM,131C,CANBERRA CRES,01 TO 03,68.0,Model A,2020-01-01,95 years 01 month,550000.0,731.9452,751.422374,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
189975,2024-09-01,SEMBAWANG,3 ROOM,132B,CANBERRA VIEW,04 TO 06,68.0,Model A,2020-01-01,95 years 01 month,530000.0,731.9452,724.097924,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
189987,2024-09-01,SEMBAWANG,4 ROOM,131B,CANBERRA CRES,04 TO 06,92.0,Model A,2020-01-01,95 years 01 month,753000.0,990.2788,760.391922,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
189988,2024-09-01,SEMBAWANG,4 ROOM,131B,CANBERRA CRES,04 TO 06,92.0,Model A,2020-01-01,95 years 01 month,760000.0,990.2788,767.460638,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190124,2024-09-01,SENGKANG,4 ROOM,455B,SENGKANG WEST AVE,07 TO 09,93.0,Model A,2020-01-01,95 years 01 month,648000.0,1001.0427,647.325034,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
190126,2024-09-01,SENGKANG,4 ROOM,458B,SENGKANG WEST RD,01 TO 03,93.0,Model A,2020-01-01,95 years 01 month,595000.0,1001.0427,594.380240,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
190405,2024-09-01,TOA PAYOH,4 ROOM,105A,BIDADARI PK DR,01 TO 03,94.0,Model A,2020-01-01,95 years,937500.0,1011.8066,926.560471,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
190407,2024-09-01,TOA PAYOH,4 ROOM,106A,BIDADARI PK DR,13 TO 15,94.0,Model A,2020-01-01,95 years,1133388.0,1011.8066,1120.162687,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


#### 1b: If there are new addresses, use the following code to select the correct address, and insert ancillary information like floor area in square feet, price per square foot, and recalculating lease commence date

In [9]:
# First, check for any missing values in the masterlist
missing_val = masterlist_df[masterlist_df.isna().any(axis=1)]
missing_val

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters


In [10]:
# Obtain the index of data points whose block + street name + postal + building information aren't present in the masterlist
for_editing_index = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'").drop_duplicates(
    subset=["town", "block", "street_name"]
).index

for_editing = df_raw.loc[for_editing_index, :]
for_editing.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft
187771,2024-08-01,SENGKANG,2 ROOM,457B,SENGKANG WEST RD,01 TO 03,38.0,2-room,2020-01-01,95 years 01 month,360000.0,409.0282,880.134915
189970,2024-09-01,SEMBAWANG,3 ROOM,131C,CANBERRA CRES,01 TO 03,68.0,Model A,2020-01-01,95 years 01 month,550000.0,731.9452,751.422374
189975,2024-09-01,SEMBAWANG,3 ROOM,132B,CANBERRA VIEW,04 TO 06,68.0,Model A,2020-01-01,95 years 01 month,530000.0,731.9452,724.097924
189987,2024-09-01,SEMBAWANG,4 ROOM,131B,CANBERRA CRES,04 TO 06,92.0,Model A,2020-01-01,95 years 01 month,753000.0,990.2788,760.391922
190124,2024-09-01,SENGKANG,4 ROOM,455B,SENGKANG WEST AVE,07 TO 09,93.0,Model A,2020-01-01,95 years 01 month,648000.0,1001.0427,647.325034


In [11]:
def obtain_lease_yearmth(row):
    today = pd.to_datetime(dt.date.today().replace(day=1))
    lease_commence = today - pd.DateOffset(
        years=row["lease_year"] + 99, months=row["lease_month"]
    )
    return lease_commence


for_editing["lease_year"] = for_editing["remaining_lease"].str.slice(0, 2).astype("int")
for_editing["lease_month"] = pd.to_numeric(
    for_editing["remaining_lease"].str.slice(9, 11), errors="coerce"
).fillna(0)

for_editing["lease_commence_date"] = for_editing.apply(obtain_lease_yearmth, axis=1)
for_editing = for_editing.drop(
    ["remaining_lease", "lease_year", "lease_month"], axis=1
)

In [12]:
for_editing.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft
187771,2024-08-01,SENGKANG,2 ROOM,457B,SENGKANG WEST RD,01 TO 03,38.0,2-room,1830-08-01,360000.0,409.0282,880.134915
189970,2024-09-01,SEMBAWANG,3 ROOM,131C,CANBERRA CRES,01 TO 03,68.0,Model A,1830-08-01,550000.0,731.9452,751.422374
189975,2024-09-01,SEMBAWANG,3 ROOM,132B,CANBERRA VIEW,04 TO 06,68.0,Model A,1830-08-01,530000.0,731.9452,724.097924
189987,2024-09-01,SEMBAWANG,4 ROOM,131B,CANBERRA CRES,04 TO 06,92.0,Model A,1830-08-01,753000.0,990.2788,760.391922
190124,2024-09-01,SENGKANG,4 ROOM,455B,SENGKANG WEST AVE,07 TO 09,93.0,Model A,1830-08-01,648000.0,1001.0427,647.325034


In [14]:
if for_editing.shape[0] > 0:
    for i in for_editing.index:
        search_value = (
            for_editing.loc[i, "block"] + " " + for_editing.loc[i, "street_name"]
        )
        print(f"{i}: {search_value}")
        response = requests.get(
            f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        )
        while response.status_code != 200:
            time.sleep(1)
            print("not 200")
            response = requests.get(
                f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
            )

        json_data = response.json()
        for j in json_data["results"]:
            print(j)
        time.sleep(0.5)
        user_input = input("Select the correct address: ")
        k = int(user_input) - 1
        print(k)

        if k == -2:
            for_editing.loc[i, "found"] = math.nan
            for_editing.loc[i, "search_val"] = math.nan
            for_editing.loc[i, "blk_no"] = math.nan
            for_editing.loc[i, "road_name"] = math.nan
            for_editing.loc[i, "building"] = math.nan
            for_editing.loc[i, "address"] = math.nan
            for_editing.loc[i, "postal"] = math.nan
            for_editing.loc[i, "x"] = math.nan
            for_editing.loc[i, "y"] = math.nan
            for_editing.loc[i, "latitude"] = math.nan
            for_editing.loc[i, "longitude"] = math.nan
        else:
            for_editing.loc[i, "found"] = 1
            for_editing.loc[i, "search_val"] = json_data["results"][k]["SEARCHVAL"]
            for_editing.loc[i, "blk_no"] = json_data["results"][k]["BLK_NO"]
            for_editing.loc[i, "road_name"] = json_data["results"][k]["ROAD_NAME"]
            for_editing.loc[i, "building"] = json_data["results"][k]["BUILDING"]
            for_editing.loc[i, "address"] = json_data["results"][k]["ADDRESS"]
            for_editing.loc[i, "postal"] = json_data["results"][k]["POSTAL"]
            for_editing.loc[i, "x"] = json_data["results"][k]["X"]
            for_editing.loc[i, "y"] = json_data["results"][k]["Y"]
            for_editing.loc[i, "latitude"] = json_data["results"][k]["LATITUDE"]
            for_editing.loc[i, "longitude"] = json_data["results"][k]["LONGITUDE"]

187771: 457B SENGKANG WEST RD
{'SEARCHVAL': 'FERNVALE WOODS', 'BLK_NO': '457B', 'ROAD_NAME': 'SENGKANG WEST ROAD', 'BUILDING': 'FERNVALE WOODS', 'ADDRESS': '457B SENGKANG WEST ROAD FERNVALE WOODS SINGAPORE 792457', 'POSTAL': '792457', 'X': '32168.7083977796', 'Y': '41592.4737811014', 'LATITUDE': '1.39242177196708', 'LONGITUDE': '103.870777707872'}
0
189970: 131C CANBERRA CRES
{'SEARCHVAL': 'EASTLINK I @ CANBERRA', 'BLK_NO': '131C', 'ROAD_NAME': 'CANBERRA CRESCENT', 'BUILDING': 'EASTLINK I @ CANBERRA', 'ADDRESS': '131C CANBERRA CRESCENT EASTLINK I @ CANBERRA SINGAPORE 753131', 'POSTAL': '753131', 'X': '27694.7150929319', 'Y': '47291.7828973755', 'LATITUDE': '1.44396456450031', 'LONGITUDE': '103.830575292056'}
0
189975: 132B CANBERRA VIEW
{'SEARCHVAL': 'EASTLINK I @ CANBERRA', 'BLK_NO': '132B', 'ROAD_NAME': 'CANBERRA VIEW', 'BUILDING': 'EASTLINK I @ CANBERRA', 'ADDRESS': '132B CANBERRA VIEW EASTLINK I @ CANBERRA SINGAPORE 752132', 'POSTAL': '752132', 'X': '27814.783099317', 'Y': '47198.4

In [15]:
for_editing_gdf = gpd.GeoDataFrame(
    for_editing, geometry=gpd.points_from_xy(for_editing["x"], for_editing["y"])
)

for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry
187771,2024-08-01,SENGKANG,2 ROOM,457B,SENGKANG WEST RD,01 TO 03,38.0,2-room,1830-08-01,360000.0,409.0282,880.134915,1.0,FERNVALE WOODS,457B,SENGKANG WEST ROAD,FERNVALE WOODS,457B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792457,32168.7083977796,41592.4737811014,1.39242177196708,103.870777707872,POINT (32168.708 41592.474)
189970,2024-09-01,SEMBAWANG,3 ROOM,131C,CANBERRA CRES,01 TO 03,68.0,Model A,1830-08-01,550000.0,731.9452,751.422374,1.0,EASTLINK I @ CANBERRA,131C,CANBERRA CRESCENT,EASTLINK I @ CANBERRA,131C CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,753131,27694.7150929319,47291.7828973755,1.44396456450031,103.830575292056,POINT (27694.715 47291.783)
189975,2024-09-01,SEMBAWANG,3 ROOM,132B,CANBERRA VIEW,04 TO 06,68.0,Model A,1830-08-01,530000.0,731.9452,724.097924,1.0,EASTLINK I @ CANBERRA,132B,CANBERRA VIEW,EASTLINK I @ CANBERRA,132B CANBERRA VIEW EASTLINK I @ CANBERRA SINGA...,752132,27814.783099317,47198.4456370585,1.44312045724646,103.831654222251,POINT (27814.783 47198.446)
189987,2024-09-01,SEMBAWANG,4 ROOM,131B,CANBERRA CRES,04 TO 06,92.0,Model A,1830-08-01,753000.0,990.2788,760.391922,1.0,EASTLINK I @ CANBERRA,131B,CANBERRA CRESCENT,EASTLINK I @ CANBERRA,131B CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,752131,27726.1310504675,47311.9097142201,1.4441465844546,103.830857595253,POINT (27726.131 47311.910)
190124,2024-09-01,SENGKANG,4 ROOM,455B,SENGKANG WEST AVE,07 TO 09,93.0,Model A,1830-08-01,648000.0,1001.0427,647.325034,1.0,FERNVALE WOODS,455B,SENGKANG WEST AVENUE,FERNVALE WOODS,455B SENGKANG WEST AVENUE FERNVALE WOODS SINGA...,792455,32109.729354891,41547.6258912347,1.39201619223955,103.870247728472,POINT (32109.729 41547.626)
190126,2024-09-01,SENGKANG,4 ROOM,458B,SENGKANG WEST RD,01 TO 03,93.0,Model A,1830-08-01,595000.0,1001.0427,594.380240,1.0,FERNVALE WOODS,458B,SENGKANG WEST ROAD,FERNVALE WOODS,458B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792458,32104.0491439952,41649.5559899863,1.39293801208567,103.870196701638,POINT (32104.049 41649.556)
190405,2024-09-01,TOA PAYOH,4 ROOM,105A,BIDADARI PK DR,01 TO 03,94.0,Model A,1830-09-01,937500.0,1011.8066,926.560471,1.0,ALKAFF VISTA,105A,BIDADARI PARK DRIVE,ALKAFF VISTA,105A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341105,32130.1762556639,35133.2631429676,1.33400699237548,103.870430572231,POINT (32130.176 35133.263)
190407,2024-09-01,TOA PAYOH,4 ROOM,106A,BIDADARI PK DR,13 TO 15,94.0,Model A,1830-09-01,1133388.0,1011.8066,1120.162687,1.0,ALKAFF VISTA,106A,BIDADARI PARK DRIVE,ALKAFF VISTA,106A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341106,32222.4509584131,35147.8612704426,1.33413899991746,103.871259715052,POINT (32222.451 35147.861)


#### 1c. Appending MRT information

In [16]:
mrt_file_loc = "../datasets/mrt_lrt_stations.csv"
mrt_df = pd.read_csv(mrt_file_loc, parse_dates=["opening"], index_col=0)
mrt_gdf = gpd.GeoDataFrame(mrt_df, geometry=gpd.points_from_xy(mrt_df["x"], mrt_df["y"]))

mrt_gdf.head(1)

,code,station_name,line,color,opening,type,blk_no,road_name,building,address,postal,x,y,latitude,longitude,planning_area_ura,region_ura,geometry
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.057052,35038.96887,1.333153,103.742286,JURONG EAST,WEST REGION,POINT (17869.057 35038.969)


In [17]:
def find_closest_station(row, mrt_gdf):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # mrt_gdf["OPENING_DATE"] = mrt_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = mrt_gdf[mrt_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    """Remove comments for this line if you want the closest MRT station today"""
    distances = mrt_gdf.distance(row["geometry"])

    closest_station_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_station_name = mrt_gdf.loc[closest_station_index, "station_name"]
    closest_transport_type = mrt_gdf.loc[closest_station_index, "type"]
    closest_mrt_color = mrt_gdf.loc[closest_station_index, "color"]

    # distance to cbd
    raffles_place_index = mrt_gdf.query("station_name == 'Raffles Place'").index[0]
    distance_to_cbd = mrt_gdf.loc[raffles_place_index, "geometry"].distance(
        row["geometry"]
    )

    return pd.Series(
        [
            closest_station_name,
            shortest_distance,
            closest_transport_type,
            closest_mrt_color,
            distance_to_cbd,
        ],
        index=[
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ],
    )


if for_editing_gdf.shape[0] > 0:
    for_editing_gdf[
        [
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ]
    ] = for_editing_gdf.apply(find_closest_station, mrt_gdf=mrt_gdf, axis=1)

In [18]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd
187771,2024-08-01,SENGKANG,2 ROOM,457B,SENGKANG WEST RD,01 TO 03,38.0,2-room,1830-08-01,360000.0,409.0282,880.134915,1.0,FERNVALE WOODS,457B,SENGKANG WEST ROAD,FERNVALE WOODS,457B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792457,32168.7083977796,41592.4737811014,1.39242177196708,103.870777707872,POINT (32168.708 41592.474),Fernvale,618.364169,LRT,Grey,12166.255411
189970,2024-09-01,SEMBAWANG,3 ROOM,131C,CANBERRA CRES,01 TO 03,68.0,Model A,1830-08-01,550000.0,731.9452,751.422374,1.0,EASTLINK I @ CANBERRA,131C,CANBERRA CRESCENT,EASTLINK I @ CANBERRA,131C CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,753131,27694.7150929319,47291.7828973755,1.44396456450031,103.830575292056,POINT (27694.715 47291.783),Canberra,138.100123,MRT,Red,17826.369114
189975,2024-09-01,SEMBAWANG,3 ROOM,132B,CANBERRA VIEW,04 TO 06,68.0,Model A,1830-08-01,530000.0,731.9452,724.097924,1.0,EASTLINK I @ CANBERRA,132B,CANBERRA VIEW,EASTLINK I @ CANBERRA,132B CANBERRA VIEW EASTLINK I @ CANBERRA SINGA...,752132,27814.783099317,47198.4456370585,1.44312045724646,103.831654222251,POINT (27814.783 47198.446),Canberra,217.240191,MRT,Red,17718.494824
189987,2024-09-01,SEMBAWANG,4 ROOM,131B,CANBERRA CRES,04 TO 06,92.0,Model A,1830-08-01,753000.0,990.2788,760.391922,1.0,EASTLINK I @ CANBERRA,131B,CANBERRA CRESCENT,EASTLINK I @ CANBERRA,131B CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,752131,27726.1310504675,47311.9097142201,1.4441465844546,103.830857595253,POINT (27726.131 47311.910),Canberra,174.694012,MRT,Red,17842.259577
190124,2024-09-01,SENGKANG,4 ROOM,455B,SENGKANG WEST AVE,07 TO 09,93.0,Model A,1830-08-01,648000.0,1001.0427,647.325034,1.0,FERNVALE WOODS,455B,SENGKANG WEST AVENUE,FERNVALE WOODS,455B SENGKANG WEST AVENUE FERNVALE WOODS SINGA...,792455,32109.729354891,41547.6258912347,1.39201619223955,103.870247728472,POINT (32109.729 41547.626),Fernvale,674.652242,LRT,Grey,12111.796394
190126,2024-09-01,SENGKANG,4 ROOM,458B,SENGKANG WEST RD,01 TO 03,93.0,Model A,1830-08-01,595000.0,1001.0427,594.380240,1.0,FERNVALE WOODS,458B,SENGKANG WEST ROAD,FERNVALE WOODS,458B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792458,32104.0491439952,41649.5559899863,1.39293801208567,103.870196701638,POINT (32104.049 41649.556),Fernvale,690.054429,LRT,Grey,12211.238236
190405,2024-09-01,TOA PAYOH,4 ROOM,105A,BIDADARI PK DR,01 TO 03,94.0,Model A,1830-09-01,937500.0,1011.8066,926.560471,1.0,ALKAFF VISTA,105A,BIDADARI PARK DRIVE,ALKAFF VISTA,105A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341105,32130.1762556639,35133.2631429676,1.33400699237548,103.870430572231,POINT (32130.176 35133.263),Potong Pasir,328.366052,MRT,Purple,5905.819506
190407,2024-09-01,TOA PAYOH,4 ROOM,106A,BIDADARI PK DR,13 TO 15,94.0,Model A,1830-09-01,1133388.0,1011.8066,1120.162687,1.0,ALKAFF VISTA,106A,BIDADARI PARK DRIVE,ALKAFF VISTA,106A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341106,32222.4509584131,35147.8612704426,1.33413899991746,103.871259715052,POINT (32222.451 35147.861),Potong Pasir,391.500224,MRT,Purple,5952.986739


#### 1d. Appending Closest School information

In [19]:
school_file_loc = "../datasets/schools_for_plotly.csv"
school_df = pd.read_csv(school_file_loc, index_col=0, dtype={"postal":"string"})
school_df["postal"] = school_df["postal"].astype("str").apply(lambda x: f"{x:0>6}")

# Convert the df into a gdf
school_gdf = gpd.GeoDataFrame(school_df, geometry=gpd.points_from_xy(school_df["x"], school_df["y"]))

In [20]:
def find_closest_school(row, school_gdf, level="PRIMARY"):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # school_gdf["OPENING_DATE"] = school_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = school_gdf[school_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    school_gdf_filtered = school_gdf.query("mainlevel_code == @level")
    """Remove comments for this line if you want the closest MRT station today"""
    distances = school_gdf_filtered.distance(row["geometry"])

    closest_school_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_school = school_gdf_filtered.loc[closest_school_index, "school_name"]

    return pd.Series(
        [
            closest_school,
            shortest_distance,
        ],
        index=[
            "closest_pri_school",
            "distance_to_pri_school_meters",
        ],
    )

for_editing_gdf[["closest_pri_school", "distance_to_pri_school_meters"]] = (
    for_editing_gdf.apply(find_closest_school, school_gdf=school_gdf, axis=1)
)

In [21]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
187771,2024-08-01,SENGKANG,2 ROOM,457B,SENGKANG WEST RD,01 TO 03,38.0,2-room,1830-08-01,360000.0,409.0282,880.134915,1.0,FERNVALE WOODS,457B,SENGKANG WEST ROAD,FERNVALE WOODS,457B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792457,32168.7083977796,41592.4737811014,1.39242177196708,103.870777707872,POINT (32168.708 41592.474),Fernvale,618.364169,LRT,Grey,12166.255411,FERNVALE PRIMARY SCHOOL,470.670567
189970,2024-09-01,SEMBAWANG,3 ROOM,131C,CANBERRA CRES,01 TO 03,68.0,Model A,1830-08-01,550000.0,731.9452,751.422374,1.0,EASTLINK I @ CANBERRA,131C,CANBERRA CRESCENT,EASTLINK I @ CANBERRA,131C CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,753131,27694.7150929319,47291.7828973755,1.44396456450031,103.830575292056,POINT (27694.715 47291.783),Canberra,138.100123,MRT,Red,17826.369114,SEMBAWANG PRIMARY SCHOOL,1067.930401
189975,2024-09-01,SEMBAWANG,3 ROOM,132B,CANBERRA VIEW,04 TO 06,68.0,Model A,1830-08-01,530000.0,731.9452,724.097924,1.0,EASTLINK I @ CANBERRA,132B,CANBERRA VIEW,EASTLINK I @ CANBERRA,132B CANBERRA VIEW EASTLINK I @ CANBERRA SINGA...,752132,27814.783099317,47198.4456370585,1.44312045724646,103.831654222251,POINT (27814.783 47198.446),Canberra,217.240191,MRT,Red,17718.494824,CHONGFU SCHOOL,999.323434
189987,2024-09-01,SEMBAWANG,4 ROOM,131B,CANBERRA CRES,04 TO 06,92.0,Model A,1830-08-01,753000.0,990.2788,760.391922,1.0,EASTLINK I @ CANBERRA,131B,CANBERRA CRESCENT,EASTLINK I @ CANBERRA,131B CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,752131,27726.1310504675,47311.9097142201,1.4441465844546,103.830857595253,POINT (27726.131 47311.910),Canberra,174.694012,MRT,Red,17842.259577,SEMBAWANG PRIMARY SCHOOL,1097.080899
190124,2024-09-01,SENGKANG,4 ROOM,455B,SENGKANG WEST AVE,07 TO 09,93.0,Model A,1830-08-01,648000.0,1001.0427,647.325034,1.0,FERNVALE WOODS,455B,SENGKANG WEST AVENUE,FERNVALE WOODS,455B SENGKANG WEST AVENUE FERNVALE WOODS SINGA...,792455,32109.729354891,41547.6258912347,1.39201619223955,103.870247728472,POINT (32109.729 41547.626),Fernvale,674.652242,LRT,Grey,12111.796394,FERNVALE PRIMARY SCHOOL,504.130558
190126,2024-09-01,SENGKANG,4 ROOM,458B,SENGKANG WEST RD,01 TO 03,93.0,Model A,1830-08-01,595000.0,1001.0427,594.380240,1.0,FERNVALE WOODS,458B,SENGKANG WEST ROAD,FERNVALE WOODS,458B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792458,32104.0491439952,41649.5559899863,1.39293801208567,103.870196701638,POINT (32104.049 41649.556),Fernvale,690.054429,LRT,Grey,12211.238236,FERNVALE PRIMARY SCHOOL,555.436648
190405,2024-09-01,TOA PAYOH,4 ROOM,105A,BIDADARI PK DR,01 TO 03,94.0,Model A,1830-09-01,937500.0,1011.8066,926.560471,1.0,ALKAFF VISTA,105A,BIDADARI PARK DRIVE,ALKAFF VISTA,105A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341105,32130.1762556639,35133.2631429676,1.33400699237548,103.870430572231,POINT (32130.176 35133.263),Potong Pasir,328.366052,MRT,Purple,5905.819506,CEDAR PRIMARY SCHOOL,603.536140
190407,2024-09-01,TOA PAYOH,4 ROOM,106A,BIDADARI PK DR,13 TO 15,94.0,Model A,1830-09-01,1133388.0,1011.8066,1120.162687,1.0,ALKAFF VISTA,106A,BIDADARI PARK DRIVE,ALKAFF VISTA,106A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341106,32222.4509584131,35147.8612704426,1.33413899991746,103.871259715052,POINT (32222.451 35147.861),Potong Pasir,391.500224,MRT,Purple,5952.986739,CEDAR PRIMARY SCHOOL,511.149664


#### 1e. Appending URA planning area and regions

In [22]:
geo_file_loc = "../datasets/sg_map/mp2014/MP14_PLNG_AREA_NO_SEA_PL.shp"
planning_areas_gdf = gpd.read_file(geo_file_loc)

planning_areas_gdf.head()

,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,ANG MO KIO,AM,N,NORTH-EAST REGION,NER,E5CBDDE0C2113055,2016-05-11,28976.8763,40229.1238,17494.240190,1.394138e+07,"POLYGON ((30658.500 42047.527, 30679.195 42020..."
1,2,BEDOK,BD,N,EAST REGION,ER,1719251260799DF6,2016-05-11,38582.6650,34032.0961,21872.798962,2.173319e+07,"POLYGON ((38974.269 36138.243, 39371.471 35747..."
2,3,BISHAN,BS,N,CENTRAL REGION,CR,BA616285F402846F,2016-05-11,28789.7630,37450.8865,13517.121556,7.618921e+06,"POLYGON ((29772.191 38311.805, 29784.826 38304..."
3,4,BOON LAY,BL,N,WEST REGION,WR,A3DC87118B43CDED,2016-05-11,13410.3824,33008.9884,18528.467448,8.279408e+06,"POLYGON ((12861.383 32207.492, 12860.555 32208..."
4,5,BUKIT BATOK,BK,N,WEST REGION,WR,FB44C870B04B7F57,2016-05-11,19255.4150,37527.6527,15234.223423,1.113326e+07,"POLYGON ((20294.455 39114.528, 20334.318 39054..."


In [23]:
for_editing_gdf = for_editing_gdf.sjoin(planning_areas_gdf[["PLN_AREA_N", "REGION_N", "geometry"]], how='left')

/Users/tim/miniforge3/envs/for_ml/lib/python3.10/site-packages/geopandas/geodataframe.py:2189: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: PROJCS["SVY21",GEOGCS["SVY21[WGS84]",DATUM["WGS_19 ...

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


In [24]:
for_editing_gdf = for_editing_gdf.rename(columns={"PLN_AREA_N":"planning_area_ura", "REGION_N":"region_ura"})

In [25]:
for_editing = for_editing_gdf[
    [
        "town",
        "block",
        "road_name",
        "blk_no",
        "street_name",
        "building",
        "postal",
        "address",
        "lease_commence_date",
        "planning_area_ura",
        "region_ura",
        "x",
        "y",
        "latitude",
        "longitude",
        "closest_mrt_station",
        "distance_to_mrt_meters",
        "transport_type",
        "line_color",
        "distance_to_cbd",
        "closest_pri_school",
        "distance_to_pri_school_meters",
    ]
]

In [26]:
for_editing

,town,block,road_name,blk_no,street_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
187771,SENGKANG,457B,SENGKANG WEST ROAD,457B,SENGKANG WEST RD,FERNVALE WOODS,792457,457B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,1830-08-01,SENGKANG,NORTH-EAST REGION,32168.7083977796,41592.4737811014,1.39242177196708,103.870777707872,Fernvale,618.364169,LRT,Grey,12166.255411,FERNVALE PRIMARY SCHOOL,470.670567
189970,SEMBAWANG,131C,CANBERRA CRESCENT,131C,CANBERRA CRES,EASTLINK I @ CANBERRA,753131,131C CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,1830-08-01,SEMBAWANG,NORTH REGION,27694.7150929319,47291.7828973755,1.44396456450031,103.830575292056,Canberra,138.100123,MRT,Red,17826.369114,SEMBAWANG PRIMARY SCHOOL,1067.930401
189975,SEMBAWANG,132B,CANBERRA VIEW,132B,CANBERRA VIEW,EASTLINK I @ CANBERRA,752132,132B CANBERRA VIEW EASTLINK I @ CANBERRA SINGA...,1830-08-01,SEMBAWANG,NORTH REGION,27814.783099317,47198.4456370585,1.44312045724646,103.831654222251,Canberra,217.240191,MRT,Red,17718.494824,CHONGFU SCHOOL,999.323434
189987,SEMBAWANG,131B,CANBERRA CRESCENT,131B,CANBERRA CRES,EASTLINK I @ CANBERRA,752131,131B CANBERRA CRESCENT EASTLINK I @ CANBERRA S...,1830-08-01,SEMBAWANG,NORTH REGION,27726.1310504675,47311.9097142201,1.4441465844546,103.830857595253,Canberra,174.694012,MRT,Red,17842.259577,SEMBAWANG PRIMARY SCHOOL,1097.080899
190124,SENGKANG,455B,SENGKANG WEST AVENUE,455B,SENGKANG WEST AVE,FERNVALE WOODS,792455,455B SENGKANG WEST AVENUE FERNVALE WOODS SINGA...,1830-08-01,SENGKANG,NORTH-EAST REGION,32109.729354891,41547.6258912347,1.39201619223955,103.870247728472,Fernvale,674.652242,LRT,Grey,12111.796394,FERNVALE PRIMARY SCHOOL,504.130558
190126,SENGKANG,458B,SENGKANG WEST ROAD,458B,SENGKANG WEST RD,FERNVALE WOODS,792458,458B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,1830-08-01,SENGKANG,NORTH-EAST REGION,32104.0491439952,41649.5559899863,1.39293801208567,103.870196701638,Fernvale,690.054429,LRT,Grey,12211.238236,FERNVALE PRIMARY SCHOOL,555.436648
190405,TOA PAYOH,105A,BIDADARI PARK DRIVE,105A,BIDADARI PK DR,ALKAFF VISTA,341105,105A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,1830-09-01,TOA PAYOH,CENTRAL REGION,32130.1762556639,35133.2631429676,1.33400699237548,103.870430572231,Potong Pasir,328.366052,MRT,Purple,5905.819506,CEDAR PRIMARY SCHOOL,603.536140
190407,TOA PAYOH,106A,BIDADARI PARK DRIVE,106A,BIDADARI PK DR,ALKAFF VISTA,341106,106A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,1830-09-01,TOA PAYOH,CENTRAL REGION,32222.4509584131,35147.8612704426,1.33413899991746,103.871259715052,Potong Pasir,391.500224,MRT,Purple,5952.986739,CEDAR PRIMARY SCHOOL,511.149664


In [27]:
new_masterlist_df = pd.concat([masterlist_df, for_editing], axis=0)
new_masterlist_df = new_masterlist_df.sort_values(by=["town", "street_name", "block"]).reset_index(drop=True)

In [28]:
new_masterlist_df.shape

(9626, 22)

In [30]:
# new_masterlist_df.to_csv(masterlist_file_loc)

### 2. Append additional information to the raw dataset

#### 2a: Reopen Masterlist

In [31]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc,
    index_col=0,
    dtype={"postal": "object"},
    parse_dates=["lease_commence_date"],
)

masterlist_df.head(1)

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254


In [32]:
masterlist_df.shape

(9626, 22)

#### 2b: Perform a left merge between the new HDB dataset and the address masterlist

In [33]:
df_new = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
)

In [34]:
df_new.query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge


In [35]:
df_new = df_new.drop("_merge", axis=1)

In [36]:
file_destination = file_loc_raw.replace("raw", "coords_mrt")
file_destination
df_new.to_csv(file_destination)

#### The following is used to call the Onemap API and populate coordinate details (deprecated, use the above masterlist instead)

In [ ]:
# # Obtain geospatial coordinates with the Onemap API

# def get_coordinates_for_each_row(row):
#     search_value = row["block"] + " " + row["street_name"]
#     # print(search_value)

#     response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")
#     while response.status_code != 200:
#         time.sleep(1)
#         print("not 200")
#         response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")

#     data = response.json()

#     row_data = data['results'][0]
#     searchval = row_data["SEARCHVAL"]
#     address = row_data["ADDRESS"]
#     postal = row_data["POSTAL"]
#     x = row_data["X"]
#     y = row_data["Y"]
#     latitude = row_data["LATITUDE"]
#     longitude = row_data["LONGITUDE"]

#     return pd.Series([address, postal, x, y, latitude, longitude], 
#                      index=["SEARCHVAL", "ADDRESS", "POSTAL", "X", "Y", "LATITUDE", "LONGITUDE"])

In [ ]:
# Appending additional columns to the dataframe


# NOTE: SHOULDNT APPEND
# for_addition["year"] = for_addition["month"].dt.year
# for_addition["lease_years"] = for_addition["remaining_lease"].str.split(" ").apply(lambda x: int(x[0]))
# bins = pd.IntervalIndex.from_tuples(
#     [(40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
# )
# for_addition["lease_cat"] = pd.cut(for_addition["lease_years"], bins)
# for_addition = for_addition.drop("index", axis=1)

#### Checking datasets

In [38]:
file_loc_raw = "../datasets/resale_hdb_price_coords_mrt_30sep.csv"
df_for_kaggle = pd.read_csv(
    file_loc_raw,
    parse_dates=["month", "lease_commence_date"],
    index_col=0,
    dtype={"x": "float64", "y": "float64", "postal": "object"},
    low_memory=False,
)

In [39]:
df_for_kaggle[df_for_kaggle.isna().any(axis=1)]
# df_for_kaggle.shape

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters


In [40]:
df_for_kaggle = df_for_kaggle.drop(["town", "flat_type", "block", "street_name"], axis=1)

In [41]:
today_date = dt.date.today().strftime("%Y-%d%b").lower()
kaggle_file_loc_updated_name = f"../datasets/resale_hdb_price_for_kaggle_{today_date}.csv"
df_for_kaggle.to_csv(kaggle_file_loc_updated_name)